# Análise de Dados de Sensores de Temperatura
Neste projeto, explorei o monitoramento de sensores de temperatura em diversos ambientes por meio de um cenário hipotético de Internet das Coisas (IoT). Meu objetivo era analisar as leituras de temperatura geradas por um sensor IoT que está instalado em algum local. O problema de negócio que eu desejava resolver era garantir o controle adequado da temperatura em ambientes críticos, como salas de servidores, armazéns de produtos sensíveis à temperatura, laboratórios ou qualquer local onde o controle preciso da temperatura seja essencial.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, max, min, stddev
from pyspark.sql.window import Window
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, TimestampType
import random
import datetime
import matplotlib.pyplot as plt

In [ ]:
spark = SparkSession.builder.appName("Análise de Temperatura de Sensores").getOrCreate()

# Dados Disponíveis
Os dados que tive à disposição para análise são dados fictícios de sensores de temperatura. Esses dados incluem informações detalhadas, como data/hora das leituras, identificação única de cada sensor e as leituras de temperatura em graus Celsius. As leituras foram geradas em intervalos de 15 minutos ao longo de um mês, representando o comportamento típico de um sensor IoT instalado em um ambiente específico.

In [ ]:
esquema = StructType([
    StructField("timestamp", TimestampType(), True),
    StructField("sensor_id", StringType(), True),
    StructField("temperatura", DoubleType(), True)
])

In [ ]:
dados = []
hora_inicial = datetime.datetime(2023, 1, 1)
hora_final = datetime.datetime(2023, 1, 31)

In [ ]:
while hora_inicial <= hora_final:
    for sensor_id in range(1, 6):
        temperatura = random.uniform(18, 30)
        dados.append((hora_inicial, f"Sensor{sensor_id}", temperatura))
    hora_inicial += datetime.timedelta(minutes=15)

In [ ]:
df = spark.createDataFrame(dados, schema=esquema)

In [ ]:
df.show()

# Análise de Tendência
Para começar, analisei a tendência das leituras de temperatura ao longo do período de um mês. Acredito que essa análise pode indicar se a temperatura média esta aumentando ou diminuindo com o tempo, o que pode apontar para problemas no sistema de controle de temperatura.

A análise de tendência é representada em um gráfico de barras, permitindo-me visualizar a temperatura média por sensor. 

In [ ]:
df_tendencia = df.groupBy("sensor_id").agg(avg("temperatura").alias("temperatura_media"))

In [ ]:
dados_tendencia = df_tendencia.toPandas()
plt.figure(figsize=(12, 6))
plt.bar(dados_tendencia["sensor_id"], dados_tendencia["temperatura_media"])
plt.xlabel("Sensor ID")
plt.ylabel("Temperatura Média (°C)")
plt.title("Análise de Tendência")
plt.show()

# Análise de Anomalias
Outra análise importante que realizei foi a detecção de anomalias nas leituras de temperatura. Para isso, criei limites superior e inferior com base na média e no desvio padrão das leituras de temperatura para cada sensor. Leituras que estivessem fora desses limites eram consideradas anomalias.

As anomalias são destacadas em um gráfico de linhas, que inclui as leituras de temperatura, a média e os limites superior e inferior. Isso me permite identificar visualmente as leituras que estão fora dos limites aceitáveis e que podem indicar problemas nos sensores ou nas condições ambientais.

In [ ]:
janela_especifica = Window.partitionBy("sensor_id").orderBy("timestamp")
df_anomalias = df.withColumn("temperatura_media", avg("temperatura").over(janela_especifica)) \
    .withColumn("desvio_padrao_temperatura", stddev("temperatura").over(janela_especifica)) \
    .withColumn("limite_superior", F.col("temperatura_media") + 2 * F.col("desvio_padrao_temperatura")) \
    .withColumn("limite_inferior", F.col("temperatura_media") - 2 * F.col("desvio_padrao_temperatura")) \
    .filter((F.col("temperatura") > F.col("limite_superior")) | (F.col("temperatura") < F.col("limite_inferior")))

In [ ]:
dados_anomalias = df_anomalias.toPandas()
plt.figure(figsize=(12, 6))
plt.plot(dados_anomalias["timestamp"], dados_anomalias["temperatura"], label="Temperatura")
plt.plot(dados_anomalias["timestamp"], dados_anomalias["temperatura_media"], label="Média")
plt.plot(dados_anomalias["timestamp"], dados_anomalias["limite_superior"], label="Limite Superior")
plt.plot(dados_anomalias["timestamp"], dados_anomalias["limite_inferior"], label="Limite Inferior")
plt.xlabel("Data e Hora")
plt.ylabel("Temperatura (°C)")
plt.title("Análise de Anomalias")
plt.legend()
plt.show()

# Análise de Padrão Diurno
Por fim, explorei a existência de um padrão diurno nas leituras de temperatura. Para isso, agrupei as leituras por hora do dia e calculei a temperatura média para cada hora.

Essa análise foi representada em um gráfico de barras, mostrando o padrão cíclico das temperaturas ao longo do dia. Isso é fundamental para entender as variações diárias e ajustar o sistema de controle de temperatura de acordo com as necessidades.

In [ ]:
df_padrao_diurno = df.withColumn("hora", F.hour("timestamp"))
df_padrao_diurno = df_padrao_diurno.groupBy("hora").agg(avg("temperatura").alias("temperatura_media"))

In [ ]:
dados_padrao_diurno = df_padrao_diurno.toPandas()
plt.figure(figsize=(12, 6))
plt.bar(dados_padrao_diurno["hora"], dados_padrao_diurno["temperatura_media"])
plt.xlabel("Hora do Dia")
plt.ylabel("Temperatura Média (°C)")
plt.title("Análise de Padrão Diurno")
plt.show()

In [ ]:
spark.stop()

# Conclusões
Análise de Tendência: Esta análise nos permitirá identificar se a temperatura média está aumentando ou diminuindo ao longo do tempo. Isso pode indicar problemas no sistema de resfriamento que precisam ser abordados, como a necessidade de manutenção preventiva ou ajustes no controle de temperatura.

Análise de Anomalias: Ao identificar leituras de temperatura que estão fora do intervalo aceitável, podemos tomar ações imediatas para solucionar problemas, como substituir sensores defeituosos ou tomar medidas para controlar as condições ambientais.

Análise de Padrão Diurno: Entender o comportamento cíclico das temperaturas ao longo do dia pode ajudar a otimizar o sistema de resfriamento, economizando energia e ajustando o controle de temperatura de acordo com as necessidades.


# O valor que esse conhecimento me traz como um profissional em automação industrial

As análises de tendência, anomalias e padrão diurno são ferramentas essenciais para profissionais de automação industrial. Elas não apenas garantem a eficiência das operações, mas também contribuem para a redução de custos, a prevenção de falhas e a segurança dos processos. A capacidade de tomar decisões informadas com base em dados é uma vantagem inestimável na automação industrial, e essas análises desempenham um papel fundamental nesse contexto.